# WFI Reference File Pipeline (RFP) - Dark Example
This notebook shows how to use the WFI Reference File Pipeline (RFP) to create a dark reference file. The RFP will simulate dark reads that will be used as input into the Dark module. 

Additional functionality such as reading a list of files to construct the input data, and optional methods to calculate the dark are in development and only mentioned, but are not necessary to generate reference files suitable for delivery to CRDS.

In [6]:
# Imports
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta
from wfi_reference_pipeline.utilities.simulate_reads import simulate_dark_reads
from wfi_reference_pipeline.reference_types.dark.dark import Dark
import numpy as np

#### Step 1: Setup

In [17]:
# Output file path
outfile = '/grp/roman/RFP/DEV/scratch/rfp_dark_dev_file.asdf'

# Use dev meta maker
tmp = MakeDevMeta(ref_type='DARK')

# Simulate a cube of dark reads.
sim_dev_cube, sim_dev_rate_image = simulate_dark_reads(48)

Making WFI Imaging Mode (WIM) dark read cube with exposure time 3.04 seconds.


#### Step 2: Instantiate the Dark object with a cube of data

In [18]:
# Instantiate rfp Dark object
rfp_dark = Dark(meta_data=tmp.meta_dark,
                ref_type_data=sim_dev_cube,
                outfile=outfile,
                clobber=True)

#### Note that the meta object can have specific meta fields modified before sending to RFP module to instantiate. The meta data object is converted to a dictionary when the file is being written in the generate_outfile() abstract method in the ReferenceType base class. This can be seen by inspecting the meta data object where the key _use_after is an object property not yet in the correct astropy date time format.

In [19]:
det = 5  # Make the readnoise reference file for WFI05.
tmp.meta_dark.instrument_detector = 'WFI' + f"{det:02d}"
tmp.meta_dark.useafter = '2024-04-22T00:00:00.000'  # Update useafter to the new string
tmp.meta_dark

WFIMetaDark(reference_type='DARK', pedigree='DUMMY', description='For RFP Development and DMS Build Support.', author='RFP', _use_after=<Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, telescope='ROMAN', origin='STSCI', instrument='WFI', instrument_detector='WFI05', ngroups=6, nframes=8, groupgap=0, ma_table_name='High Latitude Imaging Survey', ma_table_number=1)

#### Note that the dev dark meta object has an evenly spaced MA Table. This is necessary to create a valid dark reference file matching the data model which currently has ngroups and nframes. These meta data will eventually be updated to reflect the unevenly spaced read pattern flexibility shown next. 

In [20]:
# Get rate image from data cube.
rfp_dark.make_rate_image_from_data_cube()
# Average with even spacing.
rfp_dark.make_ma_table_resampled_data(num_resultants=8, num_reads_per_resultant=6)
# Update the data quality array.
rfp_dark.update_data_quality_array()
# Save file.
rfp_dark.generate_outfile()

### Create Dark reference file with an unevely spaced read pattern
This section demonstrates how to create a Dark reference file using a simulated dark data cube and a user specified read pattern.

#### Step 3: Instantiate a new Dark object with a cube of data

In [22]:
# Instantiate rfp Dark object
rfp_dark2 = Dark(meta_data=tmp.meta_dark,
                 ref_type_data=sim_dev_cube,
                 outfile=outfile,
                 clobber=True)

#### Step 4: Then resample the cube according to the read pattern, update the data quality array and generate the file.

In [23]:
# Get rate image from data cube.
rfp_dark2.make_rate_image_from_data_cube()
# Resample and average according to read pattern.
read_pattern = [
    [1],
    [2, 3],
    [4, 5, 6, 7],
    [9, 10, 11, 12, 13, 14],
    [15, 16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25, 26],
    [28, 29, 30, 31, 32, 33],
    [37, 38, 39, 40, 41, 42],
    [44, 45, 46, 47],
    [48]
]
rfp_dark2.make_ma_table_resampled_data(read_pattern=read_pattern)
# Update the data quality array.
rfp_dark.update_data_quality_array()
# Save file.
rfp_dark.generate_outfile()